In [ ]:
import math
from graphviz import Digraph
import random

Defining the Value Object

In [ ]:
class Value:
    def __init__(self, data, _children=(), _op="", label=""):
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.label = label
        self.grad = 0.0 

    def __repr__(self):
        return f"Value(data={self.data}, \"{self.label}\")"
    
    def __add__(self, other):
        out = Value(self.data + other.data, (self, other), "+")
        return out
    
    def __mul__(self, other):
        out = Value(self.data * other.data, (self, other), "*")
        return out

Create a mathematical expression

In [ ]:
a = Value(1.0); a.label="a"
b = Value(2.0); b.label="b"

c = a + b; c.label="c"

d = Value(-3.0); d.label="d"

e = c * d; e.label="e"

f = Value(-1.0); f.label = "f"

o = e + f; o.label = "o" 

Create a Graph of the Mathematical Expression

In [ ]:
# Create a set of known nodes
# and their connections
# starting from a "root" node
def trace(root):
    nodes, edges = set(), set()

    def build(node):
        if node not in nodes:
            nodes.add(node)
        for prev in node._prev:
            edges.add((prev, node))
            build(prev)

    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})

    nodes, edges = trace(root)

    # For each Value object, create a node
    for node in nodes:
        node_id = str(id(node))
        dot.node(name=node_id, label=f"{node.label} | data {node.data:.4f} | grad {node.grad:.4f}", shape="record")

        # If this Value object was created by an operation,
        # create a node for that operation, and connect it
        # to the Value object
        if node._op:
            dot.node(name= node_id + str(node._op), label=node._op)
            dot.edge(node_id + node._op, node_id)

    # Connect a child node to its parent's operation node
    for child, parent in edges:
        dot.edge(str(id(child)), str(id(parent)) + parent._op)

    return dot

draw_dot(o)

### Perform Manual Backpropagation

$o = f + e$

$\frac{do}{do} = 1$

Hence, `o.grad = 1.0`

$\frac{do}{df}= \frac{do}{do} * \frac{do}{df} = 1 * 1 = 1$

=> `f.grad = o.grad * 1.0`

$\frac{do}{de} = \frac{do}{do} * \frac{do}{df} = 1 *1 = 1$

=> `e.grad = o.grad * 1.0`

<br>

$e = cd$

$\frac{do}{dc} = \frac{do}{de} * \frac{de}{dc}$ &emsp; *Because of the chain rule*

$\frac{do}{dc} =  1 * d$

$\frac{do}{dc} = d$

=> `c.grad = e.grad * d.data`

$\frac{do}{dd} = \frac{do}{de} * \frac{de}{dd} = 1 * c = c$

=> `d.grad = e.grad * c.data`

<br>

$c = a + b$

$\frac{do}{da} = \frac{do}{dc} * \frac{dc}{da} = d * 1 = d$

=> `a.grad = c.grad * 1.0`

$\frac{do}{db} = \frac{do}{dc} * \frac{dc}{db} = d * 1 = d$

=> `b.grad = c.grad * 1.0`

In [ ]:
o.grad = 1.0
f.grad = o.grad * 1.0
e.grad = o.grad * 1.0
c.grad = e.grad * d.data
d.grad = e.grad * c.data
a.grad = c.grad * 1.0
b.grad = c.grad * 1.0

draw_dot(o)

Yet another example of manual backpropagation with a neuron-like structure

First we create an activation function (tanh) in our Value object

In [ ]:
class Value:
    def __init__(self, data, _children=(), _op="", label=""):
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.label = label
        self.grad = 0.0 

    def __repr__(self):
        return f"Value(data={self.data}, \"{self.label}\")"
    
    def __add__(self, other):
        out = Value(self.data + other.data, (self, other), "+")
        return out
    
    def __mul__(self, other):
        out = Value(self.data * other.data, (self, other), "*")
        return out

    def tanh(self):
        # math.exp(int) does e^(int)
        t = (math.exp(2*self.data) - 1)/(math.exp(2*self.data) + 1)
        out = Value(t, (self, ), "tanh")
        return out

Then we create the expression of a Neuron

In [ ]:
# Inputs
x1 = Value(2.0, label="x1")
x2 = Value(0.0, label="x2")

# Weights
w1 = Value(-3.0, label="w1")
w2 = Value(1.0, label="w2")

# Bias
b = Value(6.8813735879195432, label="b")

# Weighted inputs
x1w1 = x1 * w1; x1w1.label="x1*w1"
x2w2 = x2 * w2; x2w2.label="x2*w2"

# Overall sum
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label="x1*w1 + x2*w2"
n = x1w1x2w2 + b; n.label="n"

# Activation function
o = n.tanh(); o.label="o"

draw_dot(o)

And now we perform the actual backpropagation

$o = tanh(n)$

$\frac{do}{do}=1$

=> `o.grad = 1.0`

$\frac{do}{dn} = \frac{do}{do} * \frac{do}{dn} = 1 * \frac{d}{dn}(tanh(n)) = 1 * (1 - tanh^2(n)) = 1 * (1 - o^2) = 1 - o^2$

=> `n.grad = o.grad * (1 - o.data**2)`

<br>

$n = x1w1x2w2 + b$

$\frac{do}{dx1w1x2w2} = \frac{do}{dn} * \frac{dn}{dx1w1x2w2} = (1 - o^2) * 1 = 1 - o^2$

=> `x1w1x2w2.grad = n.grad * 1.0`

$\frac{do}{db} = \frac{do}{dn} * \frac{dn}{db} = (1 - o^2) * 1 = 1 - o^2$

=> `b.grad = n.grad * 1.0`

<br>

$x1w1x2w2 = x1w1 + x2w2$

$\frac{do}{dx1w1} = \frac{do}{dx1w1x2w2} * \frac{dx1w1x2w2}{dx1w1} = (1 - o^2) * 1 = 1 - o^2$

=> `x1w1.grad = x1w1x2w2.grad * 1.0`

$\frac{do}{x2w2} = \frac{do}{dx1w1x2w2} * \frac{dx1w1x2w2}{x2w2} = (1 - o^2) * 1 = 1 - o^2$

=> `x2w2.grad = x1w1x2w2.grad * 1.0`

<br>

$x1w1 = x1 * w1$

$\frac{do}{dx1} = \frac{do}{dx1w1} * \frac{dx1w1}{dx1} = (1 - o^2) * w1$

=> `x1.grad = x1w1.grad * w1.data`

$\frac{do}{w1} = \frac{do}{dx1w1} * \frac{dx1w1}{dw1} = (1 - o^2) * x1$

=> `w1.grad = x1w1.grad * x1.data`

<br>

$x2w2 = x2 * w2$

$\frac{do}{dx2} = \frac{do}{dx2w2} * \frac{dx2w2}{dx2} = (1 - o^2) * w2$

=> `x2.grad = x2w2.grad * w2.data`

$\frac{do}{dw2} = \frac{do}{dx2w2} * \frac{dx2w2}{dw2} = (1 - o^2) * x2$

=> `w2.grad = x2w2.grad * x2.data`


In [ ]:
o.grad = 1.0
n.grad = o.grad * (1 - o.data**2)
x1w1x2w2.grad = n.grad * 1.0
b.grad = n.grad * 1.0
x1w1.grad = x1w1x2w2.grad * 1.0
x2w2.grad = x1w1x2w2.grad * 1.0
x1.grad = x1w1.grad * w1.data
w1.grad = x1w1.grad * x1.data
x2.grad = x2w2.grad * w2.data
w2.grad = x2w2.grad * x2.data
draw_dot(o)

So now let's actually start trying to automate this

For every parent Value object, I want to be able to call a function called "_backward"

This "_backward" function simply computes the gradients of the children nodes

If a node has no children nodes, its "_backward" function will simply do nothing

I'll add comments in the changed \_\_add\_\_ method to make this more clear

I will also add the create graph functionality inside the value object, such that we can call Value.create_graph()

In [ ]:
class Value:
    def __init__(self, data, _children=(), _op="", label=""):
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.label = label
        self.grad = 0.0
        self._backward = lambda: None


    # Supported operations
    def __add__(self, other):
        out = Value(self.data + other.data, (self, other), "+")

        # Create the _backward behavior of out
        def _backward():
            # Calculate dout/dself and dout/dother, and set the grads of self and other
            dout_dself = 1.0
            dout_dother = 1.0
            self.grad = out.grad * dout_dself # Because of the chain rule
            other.grad = out.grad * dout_dother

        # Set the _backward behavior of "out" to what it's supposed to be
        out._backward = _backward

        return out
    
    def __mul__(self, other):
        out = Value(self.data * other.data, (self, other), "*")

        # Create the behavior we expect when calling backward
        def _backward():
            # Compute the local derivatives, and set the gradients of the children nodes
            dout_dself = other.data
            dout_dother = self.data
            self.grad = out.grad * dout_dself
            other.grad = out.grad * dout_dother
        
        # Set the "_backward" behavior of out to what it's supposed to be
        out._backward = _backward
        return out
    
    def tanh(self):
        t = (math.exp(2*self.data) - 1)/(math.exp(2*self.data) + 1)
        out = Value(t, (self, ), "tanh")

        # Define the backward behavior of out
        def _backward():
            # Calculate dout/dself, and then set self.grad
            # d/dx(tanh(x)) = 1 - tanh^2(x)
            # out = tanh(self)
            dout_dself = 1 - out.data**2
            self.grad = out.grad * dout_dself

        out._backward = _backward
        # Set the backward behavior of out to what its supposed to be

        return out
    

    # Cosmetic stuff
    def __repr__(self):
        return f"Value(\"{self.label}\", data={self.data:.4f})"
    
    def create_graph(self):
        # Create a set of known nodes and edges
        def trace(root):
            nodes, edges = set(), set()

            def build(node):
                if node not in nodes:
                    nodes.add(node)
                    for prev in node._prev:
                        edges.add((prev, node))
                        build(prev)

            build(self)

            return nodes, edges
        
        def draw_dot(root):
            dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})
            nodes, edges = trace(root)

            # For each Value object, create a node
            for node in nodes:
                dot.node(name=str(id(node)), label=f"{node.label} | data {node.data:.4f} | grad {node.grad:.4f}", shape="record")

                # If this Value object was created by an operation,
                # create a node for that, and connect it to the 
                # this Value object
                if node._op:
                    dot.node(name=str(id(node)) + node._op, label=node._op)
                    dot.edge(str(id(node)) + node._op, str(id(node)))

            # For each known edge, create an edge
            for child, parent in edges:
                dot.edge(str(id(child)), str(id(parent)) + parent._op)

            return dot
        
        return draw_dot(self)

Now we re-run our expression (I think this is what is referred to as the forward pass, or something like that)

In [ ]:
# Inputs
x1 = Value(2.0, label="x1")
x2 = Value(0.0, label="x2")

# Weights
w1 = Value(-3.0, label="w1")
w2 = Value(1.0, label="w2")

# Bias
b = Value(6.8813735879195432, label="b")

# Weighted inputs
x1w1 = x1 * w1; x1w1.label="x1w1"
x2w2 = x2 * w2; x2w2.label="x2w2"

# Overall sum
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label="x1w1x2w2"
n = x1w1x2w2 + b; n.label="n"

# Activation function
o = n.tanh(); o.label="o"

And we call the backward of each node, making sure it's in the correct order

In [ ]:
o.grad = 1.0

o._backward()
n._backward()
b._backward()
x1w1x2w2._backward()
x1w1._backward()
x2w2._backward()
x1._backward()
w1._backward()
x2._backward()
w2._backward()

o.create_graph() # Just make sure to call this last, else Jupyter will not display the graph

This process is still not fully ideal, as for any root node, I want to simply be able to call a "backward" function that does all of the "_backward" calls for me, in the right order

So let's do that

Take a look at the comments in the backward function

In [ ]:
class Value:
    def __init__(self, data, _children=(), _op="", label=""):
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.label = label
        self.grad = 0.0
        self._backward = lambda: None


    # Gradient computing
    def backward(self):
        # Treat self as the root node, find out the order of "_backward" calls, and perform those calls

        # do/do = 1.0
        self.grad = 1.0

        # Perform something similar to topological sort
        # "Sort every node of a directed graph,
        # such that every node appears before
        # all of the nodes it points to"
        
        # This is useful
        # Our directed graph is simply the graph of our Neuron expression
        # We want the first nodes of a sorting of our Neuron Value objects to be the inputs and weights,
        # followed by the weighted inputs
        # followed by the sum of the weighted inputs, and the bias
        # followed by the sum of those two, the "sum of the neuron"
        # followed by the squashed sum of the neuron

        # This is kind of like doing a topological sort

        topo_sorted = []
        seen = set()

        def explore(node):
            # For each child: 
            for child in node._prev:
                # If I do know you, what should I do?
                    # Wait for the exploration of its sibling, it might have a history
                # If I don't know you, what should I do?
                if child not in seen:
                    # Take record of the child, and explore its history
                    seen.add(child)
                    explore(child)
                # Now that everything is explored, we can add this child to our topo sort
                topo_sorted.append(child)

        explore(self) # Changes the topo_sorted list to what it should be

        # Now that we have the sorted Value objects,
        # call the "_backward" of self,
        # and call the "_backward" of the Value objects in the reversed of the sorted digraph

        # for index, val in enumerate(topo_sorted):
        #     print(f"{index}: {val}")

        self._backward()

        for node in reversed(topo_sorted):
            node._backward()


    # Supported operations
    def __add__(self, other):
        out = Value(self.data + other.data, (self, other), "+")

        # Create the _backward behavior of out
        def _backward():
            # Calculate dout/dself and dout/dother, and set the grads of self and other
            dout_dself = 1.0
            dout_dother = 1.0
            self.grad = out.grad * dout_dself # Because of the chain rule
            other.grad = out.grad * dout_dother

        # Set the _backward behavior of "out" to what it's supposed to be
        out._backward = _backward

        return out
    
    def __mul__(self, other):
        out = Value(self.data * other.data, (self, other), "*")

        # Create the behavior we expect when calling backward
        def _backward():
            # Compute the local derivatives, and set the gradients of the children nodes
            dout_dself = other.data
            dout_dother = self.data
            self.grad = out.grad * dout_dself
            other.grad = out.grad * dout_dother
        
        # Set the "_backward" behavior of out to what it's supposed to be
        out._backward = _backward
        return out
    
    def tanh(self):
        t = (math.exp(2*self.data) - 1)/(math.exp(2*self.data) + 1)
        out = Value(t, (self, ), "tanh")

        # Define the backward behavior of out
        def _backward():
            # Calculate dout/dself, and then set self.grad
            # d/dx(tanh(x)) = 1 - tanh^2(x)
            # out = tanh(self)
            dout_dself = 1 - out.data**2
            self.grad = out.grad * dout_dself

        out._backward = _backward
        # Set the backward behavior of out to what its supposed to be

        return out
    
    

    # Cosmetic stuff
    def __repr__(self):
        return f"Value(\"{self.label}\", data={self.data:.4f})"
    
    def create_graph(self):
        # Create a set of known nodes and edges
        def trace(root):
            nodes, edges = set(), set()

            def build(node):
                if node not in nodes:
                    nodes.add(node)
                    for prev in node._prev:
                        edges.add((prev, node))
                        build(prev)

            build(self)

            return nodes, edges
        
        def draw_dot(root):
            dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})
            nodes, edges = trace(root)

            # For each Value object, create a node
            for node in nodes:
                dot.node(name=str(id(node)), label=f"{node.label} | data {node.data:.4f} | grad {node.grad:.4f}", shape="record")

                # If this Value object was created by an operation,
                # create a node for that, and connect it to the 
                # this Value object
                if node._op:
                    dot.node(name=str(id(node)) + node._op, label=node._op)
                    dot.edge(str(id(node)) + node._op, str(id(node)))

            # For each known edge, create an edge
            for child, parent in edges:
                dot.edge(str(id(child)), str(id(parent)) + parent._op)

            return dot
        
        return draw_dot(self)

Re-run the forward pass

In [ ]:
# Inputs
x1 = Value(2.0, label="x1")
x2 = Value(0.0, label="x2")

# Weights
w1 = Value(-3.0, label="w1")
w2 = Value(1.0, label="w2")

# Bias
b = Value(6.8813735879195432, label="b")

# Weighted inputs
x1w1 = x1 * w1; x1w1.label="x1w1"
x2w2 = x2 * w2; x2w2.label="x2w2"

# Overall sum
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label="x1w1x2w2"
n = x1w1x2w2 + b; n.label="n"

# Activation function
o = n.tanh(); o.label="o"

In [ ]:
o.backward()
o.create_graph()

The only problem with the Value object we created so far exists when we try to do something like this

In [ ]:
a = Value(3.0, label="a")
b = a + a; b.label="b"

b.backward()
print(a.grad)

You see, $b = a + a$, which is equal to $b = 2a$

Hence, $\frac{db}{da} = 2$

But, from the above, we see that `a.grad = 1.0`

This bug arises from the fact that when `b.backward()` is called, `b._backward()` is also called

`b._backward()` does `self.grad = out.grad * dout_dself`, and it does `other.grad = out.grad * dout_dother`

But both `self` and `other` are `a`

So we set `a.grad = 1.0` once with `self.grad`, and twice with `other.grad`

To fix this, we simply need to change the `=` in `self.grad = ...` to a `+=`

and the same thing for `other.grad = ...`

We'll basically do this for every operation

In [ ]:
class Value:
    def __init__(self, data, _children=(), _op="", label=""):
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.label = label
        self.grad = 0.0
        self._backward = lambda: None


    # Gradient computing
    def backward(self):
        # Treat self as the root node, find out the order of "_backward" calls, and perform those calls

        # do/do = 1.0
        self.grad = 1.0

        # Perform something similar to topological sort
        # "Sort every node of a directed graph,
        # such that every node appears before
        # all of the nodes it points to"
        
        # This is useful
        # Our directed graph is simply the graph of our Neuron expression
        # We want the first nodes of a sorting of our Neuron Value objects to be the inputs and weights,
        # followed by the weighted inputs
        # followed by the sum of the weighted inputs, and the bias
        # followed by the sum of those two, the "sum of the neuron"
        # followed by the squashed sum of the neuron

        # This is kind of like doing a topological sort

        topo_sorted = []
        seen = set()

        def explore(node):
            # For each child: 
            for child in node._prev:
                # If I do know you, what should I do?
                    # Wait for the exploration of its sibling, it might have a history
                # If I don't know you, what should I do?
                if child not in seen:
                    # Take record of the child, and explore its history
                    seen.add(child)
                    explore(child)
                # Now that everything is explored, we can add this child to our topo sort
                topo_sorted.append(child)

        explore(self) # Changes the topo_sorted list to what it should be

        # Now that we have the sorted Value objects,
        # call the "_backward" of self,
        # and call the "_backward" of the Value objects in the reversed of the sorted digraph

        # for index, val in enumerate(topo_sorted):
        #     print(f"{index}: {val}")

        self._backward()

        for node in reversed(topo_sorted):
            node._backward()


    # Supported operations
    def __add__(self, other):
        out = Value(self.data + other.data, (self, other), "+")

        # Create the _backward behavior of out
        def _backward():
            # Calculate dout/dself and dout/dother, and set the grads of self and other
            dout_dself = 1.0
            dout_dother = 1.0
            self.grad += out.grad * dout_dself # Because of the chain rule
            other.grad += out.grad * dout_dother

        # Set the _backward behavior of "out" to what it's supposed to be
        out._backward = _backward

        return out
    
    def __mul__(self, other):
        out = Value(self.data * other.data, (self, other), "*")

        # Create the behavior we expect when calling backward
        def _backward():
            # Compute the local derivatives, and set the gradients of the children nodes
            dout_dself = other.data
            dout_dother = self.data
            self.grad += out.grad * dout_dself
            other.grad += out.grad * dout_dother
        
        # Set the "_backward" behavior of out to what it's supposed to be
        out._backward = _backward
        return out
    
    def tanh(self):
        t = (math.exp(2*self.data) - 1)/(math.exp(2*self.data) + 1)
        out = Value(t, (self, ), "tanh")

        # Define the backward behavior of out
        def _backward():
            # Calculate dout/dself, and then set self.grad
            # d/dx(tanh(x)) = 1 - tanh^2(x)
            # out = tanh(self)
            dout_dself = 1 - out.data**2
            self.grad += out.grad * dout_dself

        out._backward = _backward
        # Set the backward behavior of out to what its supposed to be

        return out
    
    

    # Cosmetic stuff
    def __repr__(self):
        return f"Value(\"{self.label}\", data={self.data:.4f})"
    
    def create_graph(self):
        # Create a set of known nodes and edges
        def trace(root):
            nodes, edges = set(), set()

            def build(node):
                if node not in nodes:
                    nodes.add(node)
                    for prev in node._prev:
                        edges.add((prev, node))
                        build(prev)

            build(self)

            return nodes, edges
        
        def draw_dot(root):
            dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})
            nodes, edges = trace(root)

            # For each Value object, create a node
            for node in nodes:
                dot.node(name=str(id(node)), label=f"{node.label} | data {node.data:.4f} | grad {node.grad:.4f}", shape="record")

                # If this Value object was created by an operation,
                # create a node for that, and connect it to the 
                # this Value object
                if node._op:
                    dot.node(name=str(id(node)) + node._op, label=node._op)
                    dot.edge(str(id(node)) + node._op, str(id(node)))

            # For each known edge, create an edge
            for child, parent in edges:
                dot.edge(str(id(child)), str(id(parent)) + parent._op)

            return dot
        
        return draw_dot(self)

Now backpropagating through `b = a + a` should work, resulting in `a.grad = 2.0`

In [ ]:
a = Value(3.0, label="a")
b = a + a; b.label="b"

b.backward()

print(a.grad)

We are now going to try and break up `tanh` into its individual operations

You see, $tanh(x) = (e^{(2x)} - 1) / (e^{(2x)} - 1)$

But we are going to do so by doing the most "atomic" operations possible

We need to implement Division of Value objects, Exponentiating a Value object to an int/float, Exponentiating "e" to a Value object, Multiplication of a Value object by a constant, and Subtraction of Value objects

In [ ]:
class Value:
    def __init__(self, data, _children=(), _op="", label=""):
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.label = label
        self.grad = 0.0
        self._backward = lambda: None



    # Operations 
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), "+")

        # Create the _backward behavior of out
        def _backward():
            # Calculate dout/dself and dout/dother, and set the grads of self and other
            dout_dself = 1.0
            dout_dother = 1.0
            self.grad += out.grad * dout_dself # Because of the chain rule
            other.grad += out.grad * dout_dother

        # Set the _backward behavior of "out" to what it's supposed to be
        out._backward = _backward

        return out
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), "*")

        # Create the behavior we expect when calling backward
        def _backward():
            # Compute the local derivatives, and set the gradients of the children nodes
            dout_dself = other.data
            dout_dother = self.data
            self.grad += out.grad * dout_dself
            other.grad += out.grad * dout_dother
        
        # Set the "_backward" behavior of out to what it's supposed to be
        out._backward = _backward
        return out
    
    def __neg__(self):
        return self * -1
    
    def __sub__(self, other):
        return self + (-other)

    def __rmul__(self, other):
        return self * other 

    def __pow__(self, other):
        assert isinstance(other, (int, float))
        out = Value(self.data**other, (self,), f"^{other}")

        def _backward():
            dout_dself = other * self.data**(other - 1)
            self.grad += out.grad * dout_dself

        out._backward = _backward

        return out

    def __truediv__(self, other):
        return self * other**-1

    def exp(self):
        out = Value(math.exp(self.data), (self, ), "e^")

        def _backward():
            dout_dself = out.data
            self.grad += out.grad * dout_dself

        out._backward = _backward

        return out
        


    # Backprog stuff 
    def backward(self):
        # Treat self as the root node, find out the order of "_backward" calls, and perform those calls

        # do/do = 1.0
        self.grad = 1.0

        # Perform something similar to topological sort
        # "Sort every node of a directed graph,
        # such that every node appears before
        # all of the nodes it points to"
        
        # This is useful
        # Our directed graph is simply the graph of our Neuron expression
        # We want the first nodes of a sorting of our Neuron Value objects to be the inputs and weights,
        # followed by the weighted inputs
        # followed by the sum of the weighted inputs, and the bias
        # followed by the sum of those two, the "sum of the neuron"
        # followed by the squashed sum of the neuron

        # This is kind of like doing a topological sort

        topo_sorted = []
        seen = set()

        def explore(node):
            # For each child: 
            for child in node._prev:
                # If I do know you, what should I do?
                    # Wait for the exploration of its sibling, it might have a history
                # If I don't know you, what should I do?
                if child not in seen:
                    # Take record of the child, and explore its history
                    seen.add(child)
                    explore(child)
                # Now that everything is explored, we can add this child to our topo sort
                    topo_sorted.append(child)


        explore(self) # Changes the topo_sorted list to what it should be

        # Now that we have the sorted Value objects,
        # call the "_backward" of self,
        # and call the "_backward" of the Value objects in the reversed of the sorted digraph

        # for index, val in enumerate(topo_sorted):
        #     print(f"{index}: {val}")

        self._backward()

        for node in reversed(topo_sorted):
            node._backward()



    # Cosmetic stuff
    def __repr__(self):
        return f"Value(\"{self.label}\", data={self.data:.4f})"
    
    def create_graph(self):
        # Create a set of known nodes and edges
        def trace(root):
            nodes, edges = set(), set()

            def build(node):
                if node not in nodes:
                    nodes.add(node)
                    for prev in node._prev:
                        edges.add((prev, node))
                        build(prev)

            build(self)

            return nodes, edges
        
        def draw_dot(root):
            # graph_attr={'rankdir': 'LR'}
            dot = Digraph(format='svg')
            nodes, edges = trace(root)

            # For each Value object, create a node
            for node in nodes:
                dot.node(name=str(id(node)), label=f"{node.label} | data {node.data:.4f} | grad {node.grad:.4f}", shape="record")

                # If this Value object was created by an operation,
                # create a node for that, and connect it to the 
                # this Value object
                if node._op:
                    dot.node(name=str(id(node)) + node._op, label=node._op)
                    dot.edge(str(id(node)) + node._op, str(id(node)))

            # For each known edge, create an edge
            for child, parent in edges:
                dot.edge(str(id(child)), str(id(parent)) + parent._op)

            return dot
        
        return draw_dot(self)

Now we should be able to do a couple of cool stuff

In [ ]:
a = Value(2.0, label="a")
b = Value(4.0, label="b")

c = b / a; c.label="c"

d = 2*c; d.label="d"

e = d + d

# e^(2d), where "e" is Euler's number
f = e.exp(); f.label="f"

f.backward(); f.create_graph()

In fact, we should be able to build out our Neuron expression without using the Value.tanh() method

In [ ]:
# Just building out the expression for a Neuron
x1 = Value(2.0, label="x1")
x2 = Value(0.0, label="x2")

w1 = Value(-3.0, label="w1")
w2 = Value(1.0, label="w2")

x1w1 = x1*w1; x1w1.label="x1*w1"
x2w2 = x2*w2; x2w2.label="x2*w2"

x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = "x1*w1 + x2*w2"

b = Value(6.8813735870195432, label="b")

n = x1w1x2w2 + b; n.label="n"

two_n = 2*n; two_n.label="2*n"

e = two_n.exp(); e.label = "e^(2*n)"

e_minus_1 = e - 1; e_minus_1.label = "e - 1"

e_plus_1 = e + 1; e_plus_1.label = "e + 1"

o = e_minus_1 / e_plus_1; o.label = "o"

o.backward()
o.create_graph()

So, the reason the code above does not work is due to my bad implementation of a topological sort

Turns out that my `explore()` function, which is an inner function of the Value.backward(), is just straight up broken

Here's the code for that

``` python
def explore(node):
    # For each child: 
    for child in node._prev:
        # If I do know you, what should I do?
            # Wait for the exploration of its sibling, it might have a history
        # If I don't know you, what should I do?
        if child not in seen:
            # Take record of the child, and explore its history
            seen.add(child)
            explore(child)
        # Now that everything is explored, we can add this child to our topo sort
        topo_sorted.append(child)
```

You can see the problem with this kind of approach: we allow ourselves to add nodes that we have potentially already seen

For instance, we add the node `e` twice to our list

We add it once whilst exploring the children nodes of `e - 1`

And we add it a second time whilst exploring the children nodes of `e + 1`

Hence, we end up calling `e._backward()` twice, which is not what we want

To fix this problem, we simply need to move `topo_sorted.append(child)` to the inner `if` statement

Like so: 

``` python
def explore(node):
    # For each child: 
    for child in node._prev:
        # If I do know you, what should I do?
            # Wait for the exploration of its sibling, it might have a history
        # If I don't know you, what should I do?
        if child not in seen:
            # Take record of the child, and explore its history
            seen.add(child)
            explore(child)
            # Now that everything is explored, we can add this child to our topo sort
            topo_sorted.append(child)
```

What I'll do now is that change, plus some cleaning up of the Value class

I will also change the `explore` algorithm to the below:

``` python
def explore(node):
            if node not in seen:
                seen.add(node)
                for child in node._prev:
                    explore(child)
                topo_sorted.append(node)
```

The previous explore algorithm I showed was a creation of my own, and clearly I made some mistakes

This is the "documented" explore algorithm, and so it is certain to work

I will dwell more on my implementation of this task compared to the "correct" implementation later on - who knows, maybe I'll find out that my implementation is completely valid, or that it is wrong in some other manner

Further, I will also implement __radd__ and __rsub__, in case we need to do some constant +- Value object operation

In [ ]:
class Value:
    def __init__(self, data, _children=(), _op="", label="", special_label=""):
        self.data = data
        self.label = label + special_label
        self._prev = set(_children)
        self._op = _op
        self.grad = 0.0
        self._backward = lambda: None



    # Operations
    # Value + Value
    def __add__(self, other):
        # If other is a Value object, keep it
        # Else (if other is just a int/float, for instance), create a Value object based on other
        other = other if isinstance(other, Value) else Value(other)

        out = Value(self.data + other.data, (self, other), "+")
    
        # Create out's _backward behavior
        def _backward():
            # Compute the local derivatives of out with respect to self and other
            # and change self's and other's grads
            dout_dself = 1.0
            dout_dother = 1.0
            self.grad += out.grad * dout_dself
            other.grad += out.grad * dout_dother

        # Set out's _backward behavior to what it should be
        out._backward = _backward

        return out

    def __radd__(self, other):
        return self + other

    # Value * Value
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), "*")

        def _backward():
            dout_dself = other.data
            dout_dother = self.data
            self.grad += out.grad * dout_dself
            other.grad += out.grad * dout_dother

        out._backward = _backward

        return out

    # -Value
    def __neg__(self):
        # self.__mul__(-1)
        return self * -1
    
    # Value - Value
    def __sub__(self, other):
        # self.__add__(other.__neg__)
        return self + (-other)
    
    def __rsub__(self, other):
        return self - other

    # If python cannot do other * self (for instance, when "other" is an int/float),
    # then this function gets called
    # as in 2 * Value
    def __rmul__(self, other):
        return self * other

    # (Value)^int
    def __pow__(self, other):
        # Make sure that other is a float or int
        assert isinstance(other, (int, float))
        
        out = Value(self.data**other, (self, ), f"^{other}")

        def _backward():
            dout_dself = other * self.data**(other - 1)
            self.grad += out.grad * dout_dself

        out._backward = _backward

        return out

    # Value / Value
    def __truediv__(self, other):
        # self.__mul__(other.__pow__(-1))
        return self * other**-1

    # e^(Value)
    def exp(self):
        out = Value(math.exp(self.data), (self, ), "e^")
        
        def _backward():
            dout_dself = out.data
            self.grad += out.grad * dout_dself

        out._backward = _backward

        return out

    # tanh(Value)
    def tanh(self):
        out = Value(math.tanh(self.data), (self, ) , "tanh")

        def _backward():
            dout_dself = 1 - out.data**2
            self.grad += out.grad * dout_dself

        out._backward = _backward

        return out



    # Backprog stuff
    def backward(self):
        # Treat self as a root node
        # dself/dself = 1.0, hence
        self.grad = 1.0

        # Create a list of the sorted nodes of a digraph
        # where the digraph is simply 
        # the graph of the expression of the Value objects leading up to self
        topo_sorted = []
        seen = set()

        def explore(node):
            if node not in seen:
                seen.add(node)
                for child in node._prev:
                    explore(child)
                topo_sorted.append(node)

        # This updates our "topo_sorted" and "seen" local variables

        explore(self) 

        # For every node in our reversed, topologically-sorted digraph,
        # call its _backward behavior.
        # reversed because we want to call the leaf nodes last,
        # and the leaf nodes are the nodes with least in-degrees (topo-sort jargon) 
        for node in reversed(topo_sorted): 
            node._backward()



    # Cosmetic stuff
    def __repr__(self):
        return f"Value(\"{self.label}\", data={self.data})"

    def create_graph(self):
        # Create a set of known edges (connections between Value objects) 
        # and known nodes (known Value objects)
        def trace(root):
            nodes, edges = set(), set()

            def build(node):
                if node not in nodes:
                    nodes.add(node)
                    for child in node._prev:
                        edges.add((child, node))
                        build(child)

            build(root)
            return nodes, edges

        def draw_dot(root):
            nodes, edges = trace(root)

            dot = Digraph(format="svg", graph_attr={'rankdir': 'LR'})

            # For each Value object, create a node for it
            for node in nodes:
                dot.node(name=str(id(node)), label=f"{node.label} | data {node.data:.4f} | grad {node.grad:.4f}", shape="record")

                # If this node was created by an operation, we must create a node for that too
                # and then connect that operation node to the Value object node
                if node._op:
                    dot.node(name=str(id(node)) + node._op, label=node._op)
                    dot.edge(str(id(node)) + node._op, str(id(node)))

            # For each known connection between two Value objects
            for child, parent in edges:
                # Connect the child to the parent's operation node
                dot.edge(str(id(child)), str(id(parent))+parent._op)

            return dot

        graph = draw_dot(self)
        # graph.render(directory="graphviz_outs")
        return graph

Now we do the forward pass again, do the backward, and (hopefully) get a better looking expression graph!

In [ ]:
# Just building out the expression for a Neuron
x1 = Value(2.0, label="x1")
x2 = Value(0.0, label="x2")

w1 = Value(-3.0, label="w1")
w2 = Value(1.0, label="w2")

x1w1 = x1*w1; x1w1.label="x1*w1"
x2w2 = x2*w2; x2w2.label="x2*w2"

x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = "x1*w1 + x2*w2"

b = Value(6.8813735870195432, label="b")

n = x1w1x2w2 + b; n.label="n"

two_n = 2*n; two_n.label="2*n"

e = two_n.exp(); e.label = "e^(2*n)"

e_minus_1 = e - 1; e_minus_1.label = "e - 1"

e_plus_1 = e + 1; e_plus_1.label = "e + 1"

o = e_minus_1 / e_plus_1; o.label = "o"

o.backward()
o.create_graph()

Awesome

Now let's actually
### Build out a MULTILAYERPERCEPTRON

In [ ]:
class Neuron:
    def __init__(self, n_inputs, layer_number="", neuron_number=""): 

        # Each Neuron has n weights

        self.weights = [Value(random.uniform(-1, 1), label=f"Weight {i} of Neuron {neuron_number} from Layer {layer_number}") for i in range(n_inputs)]
        # self.weights = [Value(1.0, label=f"Weight {i} of Neuron {neuron_number} from Layer {layer_number}") for i in range(n_inputs)]

        # Each Neuron is also associated with a bias

        self.bias = Value(random.uniform(-1, 1), label=f"Bias of Neuron {neuron_number} from Layer {layer_number}")
        # self.bias = Value(2.0, label=f"Bias of Neuron {neuron_number} from Layer {layer_number}")

        self.layer_number = layer_number
        self.neuron_number = neuron_number
        
    def __call__(self, inputs):
        # Given a set of n inputs,
        # multiply the ith_input with the ith_weight for every ith input and weight,
        # add together all of the ith_weighted_inputs,
        # add the bias on top,
        # and return the squashed sum
        activation = sum((ith_weight*ith_input for ith_weight, ith_input in zip(self.weights, inputs)), self.bias)
        out = activation.tanh()
        out.label=f"Output of Neuron {self.neuron_number} from Layer {self.layer_number}"
        # return activation
        return out

    def __repr__(self):
        output = f"Neuron({len(self.weights)}w="

        ws = []
        for w in self.weights:
            ws.append(w)
        ws = [f"{w.data:.4f}" for w in ws]
        ws = ", ".join(ws)

        output = output + f"[{ws}]" + f", b={self.bias.data:.4f})"

        return output
   
class Layer:
    def __init__(self, n_inputs_per_neuron, n_neurons, layer_number=""):
        self.neurons = [Neuron(n_inputs_per_neuron, layer_number=layer_number, neuron_number=i) for i in range(n_neurons)]
        self.dimensionality = n_inputs_per_neuron
        
    def __call__(self, inputs):
        # Given a set of n inputs,
        # evaluate every Neuron in this Layer for those inputs
        outs = [neuron(inputs) for neuron in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def __repr__(self):
        output = f"Layer(neurons={len(self.neurons)}, d={self.dimensionality}):\n"

        ns = []
        for index, neuron in enumerate(self.neurons):
            ns.append(f"\t{index}: {neuron}")
        ns = "\n".join(ns)
        
        return output + ns

class MLP:
    def __init__(self, n_inputs_per_neuron, n_neurons_per_layer: list):

        # Produces a list of [n_inputs_per_neuron, n_neurons_per_layer[0], n_neurons_per_layer[1], n_neurons_per_layer[2], ...]
        sz = [n_inputs_per_neuron] + n_neurons_per_layer

        n_layers = len(n_neurons_per_layer)
        self.layers = [Layer(n_inputs_per_neuron=sz[i], n_neurons=sz[i+1], layer_number=i) for i in range(n_layers)]

    def __call__(self, inputs):
        # Given an initial set of inputs,
        # evaluate the first layer for the first set of inputs,
        # use the output of that as the inputs for the second layer, evaluate it,
        # use the output of the second layer evaluated by the outputs of the first layer evaluated by the inputs
        # as
        # the input for the evaluation of the third layer
        # so on, so forth
        for layer in self.layers:
            inputs = layer(inputs)

        # Remember, if a Neuron is given less inputs than its weights,
        # then it simply ignores the weights which have no inputs attribute to during the calculation of its activation
        # However, for every Layer in an MLP, every Neuron of that Layer has
        # the same number of inputs as
        # the number of Neurons of the previous Layer
        # Hence, this situation of "Neurons-lacking inputs" never actually happens

        # Return the evaluation of the last Layer
        return inputs
    
    def __repr__(self):
        output = f"MLP(layers={len(self.layers)})\n"
        ls = []
        for layer in self.layers:
            ls.append(f"{layer}")
        ls = "\n".join(ls)
        return output + ls
    
# Whenever a Neuron does not receive the 
# same amount of inputs as its amount of weights, 
# it just ignores the trailing weight(s) in the calculation of its activation
def test0():
    inputs = [1.0, 2.0, 3.0]
    n = Neuron(4, 0, 0)
    print(n)
    
    out = n(inputs) 
    return out

# Building out a MLP
def test1():
    inputs = [3.0, 2.0, 1.0]
    mlp = MLP(len(inputs), [4, 4, 1])

    return mlp(inputs)

out = test1()
out.create_graph()

So that is quite a lot

Let me try to explain all that

Let's start, once again, with the idea of a 
#### Neuron

A Neuron consists of 4 key elements: Inputs, Weights, a Bias, and an Activation Function

Every Input enters a Neuron through its assigned gate. In every gate, lies the assigned Weight of that Gate

To cross the gate, the Input must multiply itself with the Weight that lies in that gate

This process happens for every Input.

All of these Inputs multiplied with their respective Weights - all of these _Weighted Inputs_ - make their way to the center of the Neuron

Once all gathered, the Weighted Inputs become one (we sum all of the Weighted Inputs together)

They become a single unit - **the Activation** 

Before leaving the Neuron, **the Activation** is treated to one more process: the Activation **Function**

the Activation **Function** squishes and morphs **the Activation**, twisting it into a predifined format that other Neurons find acceptable

This **Function** turns **the Activation** into an Output

Then, and only then, does the Output leave the Neuron, ready to embark in its other adventure

Alright, so now we have the notion of a Neuron. Let's now understand what a

#### Layer of Neurons

looks like

Quite simply, a Layer of Neurons is just a bunch of Neurons grouped together

These Neurons each have their own Biases, Weights, and activation Functions

But they all share one commonality: They all have the same number of Inputs

So what does it mean to give a Layer a set of Inputs?

Well, think of the Layer as a Mother with the power of Duplication

The children are the Neurons, and they are hungry

The Mother is given a set of Inputs, and it distributes those Inputs equally for each one of its children Neurons

Each child receives the same set of Inputs, since the Mother has cloning powers

The Neurons then eat the Inputs (in the fashion that we described above).

Nurished, each Neuron works to produce their very own Output and, in appreciation for their Mother's care, they each give their Output to their Mother

The Mother, then, finds herself with "n" Outputs, one for each of its Neurons

And it returns those Outputs to whoever gave her the Inputs

And can you guess who originally gave her (the Mother/Layer) those Inputs?

#### THE MULTILAYERPERCEPTRON

It actually not as Evil as it sounds. If anything, the Multi-layer Perceptron, or MLP for short, is actually a very caring person, and quite concerned about its community

In fact, it's actually easier to think of the MLP as an organization (perhaps a nonprofit one) that distributes food to those in need

Amongst these people are our Mothers (or our Layers)

The MLP is given an initial funding (the initial set of inputs), which it gives to Mother 1

Mother 1, then, gives a unique copy of those Inputs to each of its children (the Neurons), generating a set of Outputs, which the Mother gives back to the MLP

(See, the children are giving back to the community!)

The MLP then reinvests those Inputs by giving it to Mother 2. Mother 2 generates its Outputs, which it gives back to the MLP

The MLP then gives those Outputs as Inputs to Mother 3, which generates its outputs, and so soon, so forth

There's just one 
#### Caveat
To this process

Remember how the only commonality between Neurons of the same Layer is their Number of Inputs?

Well, so that actually matters in our process here.

Let us say that Mother 1 has 10 Children, and that each of these Children have a total of 5 mouths (5 Inputs). An abomination, I know!

Mother 1 will only be able to feed its Children (and, hence, generate its Outputs) if it is supplied with an Initial set of Inputs which contains 5 elements

If the Input to the Mother contains 6 or 4 elements, we'll either have-left over food, or the Children will be malnourished, either scenario being unacceptable!

Further still, if Mother 1 has 10 Children, that means that it generates an Output of 10 elements.

Hence, if Mother 2 has 11 Children, this means that if we give Mother 1's Outputs as Inputs to Mother 2, Mother 2 will find herself with hungry children!

So the MLP has a new job to perform: Make sure that its initial funding is enough to feed Mother 1's children, and make sure that Mother 1's Outputs are correspondent with the hungriness of Mother 2's children

Or, in other words, the number of Initial Inputs to a MLP must equal to the number of Inputs that First-Layer-type Neurons have, and the Number of Neurons in any layer must equal the Number of Inputs that Neurons of the subsequent layer have __(such that the Number of Neurons of the First Layer equal the Number of Inputs of Second-Layer-type Neurons, and the Number of Neurons of the Second Layer equal the Number of Inputs of Third-Layer-type Neurons, and so on, so forth)__

Please also note that Number of Neurons in any Layer does not have to equal to the Number of Neurons in the Previous Layer. All that matters is that the Number of Inputs for the Neurons of a Layer equal to the number of Neurons in the preceeding Layer.

---

And that is essentially it! That is all you need to know to intuitivelly understand an MLP (without all of the adjusting weights, biases, and backprog stuff)

So why don't we start tinkering with those latter concepts?

Let's establish the following: we have a neural network which takes 3 inputs, and outputs a single value

Let's write down four sets of Inputs, where each input contains 4 elements...

In [ ]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]

And then a set of what our desired outputs are.

Since we have four sets of inputs, we should have a set of four desired outputs, one output for each set of inputs

In [ ]:
ys = [1.0, -1.0, -1.0, 1.0]

Then, we then define our Multi-layer Perceptron.

Our Initial set of Inputs contains 3 elements.

Hence, the Number of Inputs for First-Layer-type Neurons of our MLP should be 3

Let's make a First Layer with 4 3-input Neurons, a Second Layer with 4 4-input Neurons, and a Final Layer with 1 4-input Neuron

In [ ]:
mlp = MLP(3, [4, 4, 1])

See how easy that was?!

Now we should just be able to do `mlp(xs[0])` and get a result

That result, in this context, we'll interpret as the prediction for `ys[0]`

That is, **evaluating our Set of Inputs with our Model yields a Prediction of our Set of Outputs**

In [ ]:
y0_pred = mlp(xs[0])
print(y0_pred)

In fact, let's do this for every `xs` that we have

In [ ]:
ypred = [mlp(xi) for xi in xs]
for ygt, yout in zip(ys, ypred):
    print(yout, "Expected:", ygt)

So, we can see right now that our model is not performing very well.

Indeed, when we expect it to output 1.0, it outputs approximately 0.2894, and when we expect it to output -1.0, it outputs approximately 0.0358

(Please note that the numbers 0.2894 and 0.0358 are random, and will change if you run the code above again. Nevertheless, it is most likely that our model is not performing well for random weights and biases)

In fact, **for every version of our model, we would like to qualify how well it is doing**

For this purpose, we'll create what we call the **Loss Function**

The **Loss Function** simply takes the sum of the squared __differences between the experimental outputs and the expected outputs__

In [ ]:
loss_terms = [(ygt - yout)**2 for ygt, yout in zip(ys, ypred)]
for index, loss_term in enumerate(loss_terms):
    loss_term.special_label = f"Loss of Output {index}"
    loss_term.label = loss_term.label + loss_term.special_label
loss_terms

Our prediction for our first set of inputs, namely `ypred[0]`, was close to the expected output, `ys[0]`

Hence, it should come as no surprise that our loss for predicting approximately 0.7265 (`ypred[0]`) for the expected value of 1.0 (`ys[0]`) generates a low value of approximately 0.0748 (`(ys[0]-ypred[0])**2`)

However, predicting 0.0358 for -1.0 is not good, and hence the corresponding loss value is much higher, at 1.0728

In [ ]:
# The ideal way to compute the loss is by doing this...
# loss = sum((ygt - yout)**2 for ygt, yout in zip(ys, ypred))
# But I want to generate a labelled graph.
# Hence,
loss = loss_terms[0] + loss_terms[1] + loss_terms[2] + loss_terms[3]
loss.label = "loss"; loss 

And here's the cool thing about this loss: we can backpropagate through it, and graph it too


In [ ]:
loss.backward()
loss.create_graph()

So you see, the cool thing now is that we are able to gather the grad of weights

We are able to see how the weights of our Neural Network influence the Loss Function

For instance, say that we want to decrease the final Loss value, and that a weight of a particular neuron in a particular layer is negative. 

This means that if we increase that particular weight of that particular neuron of that particular layer, we expect our loss function to get more negative

Since we want to decrease our final loss value, increasing that particular weight should help us do that

Let's create a `parameters()` method that should allow us to collect that weights and biases of a Neuron

In [ ]:
class Neuron:
    def __init__(self, n_inputs, layer_number="", neuron_number=""): 

        # Each Neuron has n weights

        self.weights = [Value(random.uniform(-1, 1), label=f"Weight {i} of Neuron {neuron_number} from Layer {layer_number}") for i in range(n_inputs)]
        # self.weights = [Value(1.0, label=f"Weight {i} of Neuron {neuron_number} from Layer {layer_number}") for i in range(n_inputs)]

        # Each Neuron is also associated with a bias

        self.bias = Value(random.uniform(-1, 1), label=f"Bias of Neuron {neuron_number} from Layer {layer_number}")
        # self.bias = Value(2.0, label=f"Bias of Neuron {neuron_number} from Layer {layer_number}")

        self.layer_number = layer_number
        self.neuron_number = neuron_number
        
    def __call__(self, inputs):
        # Given a set of n inputs,
        # multiply the ith_input with the ith_weight for every ith input and weight,
        # add together all of the ith_weighted_inputs,
        # add the bias on top,
        # and return the squashed sum
        activation = sum((ith_weight*ith_input for ith_weight, ith_input in zip(self.weights, inputs)), self.bias)
        out = activation.tanh()
        out.label=f"Output of Neuron {self.neuron_number} from Layer {self.layer_number}"
        # return activation
        return out
    
    def parameters(self):
        return self.weights + [self.bias]

    def __repr__(self):
        output = f"Neuron({len(self.weights)}w="

        ws = []
        for w in self.weights:
            ws.append(w)
        ws = [f"{w.data:.4f}" for w in ws]
        ws = ", ".join(ws)

        output = output + f"[{ws}]" + f", b={self.bias.data:.4f})"

        return output
    
class Layer:
    def __init__(self, n_inputs_per_neuron, n_neurons, layer_number=""):
        self.neurons = [Neuron(n_inputs_per_neuron, layer_number=layer_number, neuron_number=i) for i in range(n_neurons)]
        self.dimensionality = n_inputs_per_neuron
        
    def __call__(self, inputs):
        # Given a set of n inputs,
        # evaluate every Neuron in this Layer for those inputs
        outs = [neuron(inputs) for neuron in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

    def __repr__(self):
        output = f"Layer(neurons={len(self.neurons)}, d={self.dimensionality}):\n"

        ns = []
        for index, neuron in enumerate(self.neurons):
            ns.append(f"\t{index}: {neuron}")
        ns = "\n".join(ns)
        
        return output + ns

class MLP:
    def __init__(self, n_inputs_per_neuron, n_neurons_per_layer: list):

        # Produces a list of [n_inputs_per_neuron, n_neurons_per_layer[0], n_neurons_per_layer[1], n_neurons_per_layer[2], ...]
        sz = [n_inputs_per_neuron] + n_neurons_per_layer

        n_layers = len(n_neurons_per_layer)
        self.layers = [Layer(n_inputs_per_neuron=sz[i], n_neurons=sz[i+1], layer_number=i) for i in range(n_layers)]

    def __call__(self, inputs):
        # Given an initial set of inputs,
        # evaluate the first layer for the first set of inputs,
        # use the output of that as the inputs for the second layer, evaluate it,
        # use the output of the second layer evaluated by the outputs of the first layer evaluated by the inputs
        # as
        # the input for the evaluation of the third layer
        # so on, so forth
        for layer in self.layers:
            inputs = layer(inputs)

        # Remember, if a Neuron is given less inputs than its weights,
        # then it simply ignores the weights which have no inputs attribute to during the calculation of its activation
        # However, for every Layer in an MLP, every Neuron of that Layer has
        # the same number of inputs as
        # the number of Neurons of the previous Layer
        # Hence, this situation of "Neurons-lacking inputs" never actually happens

        # Return the evaluation of the last Layer
        return inputs

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]
    
    def __repr__(self):
        output = f"MLP(layers={len(self.layers)})\n"
        ls = []
        for layer in self.layers:
            ls.append(f"{layer}")
        ls = "\n".join(ls)
        return output + ls   

Re-init the MLP

In [ ]:
n_inputs = 3
mlp = MLP(n_inputs, [4, 4, 1])

And collect its parameters

In [ ]:
print("Total of", len(mlp.parameters()), "parameters")
mlp.parameters()

In [ ]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0]
ypred = [mlp(xi) for xi in xs]
loss = sum((ygt - yout)**3 for ygt, yout in zip(ys, ypred))
print("Current loss is", loss.data)

Now that we have the power to collect hte parameters of our MLP, let's change them

Remember, all these parameters are just `Value` object which represent the Weights and Biases of Neurons

Each one of them contains a `Value.data` and a `Value.grad`

Say that a Value has a negative grad

Increasing that Value's data decreases the loss

`Value.data += (-0.01) * Value.grad`

Decreasing that Value's data increases the loss

`Value.data += 0.01 * Value.grad`

<br>

Now say that a Value has a positive grad

Increasing that Value's data increases the loss

`Value.data += 0.01 * Value.grad`

Decreasing that Value's data decreases the loss

`Value.data += (-0.01) * Value.grad`

<br>

So it's clear from the above that, in order to tune our Neural Network to minimize our loss, we must

`Value.data += (-0.01) * Value.grad`

for every parameter

It's just a matter of recognizing that "**-**" negative sign, and the nuance of that with `grad`\`s being positive or negative

Another way to think about this is thinking of `grad` as a vector that points in the direction of increasing the loss

And, since we want to decrease the loss, we must go in the opposite direction of that vector - we must include the "**-**" negative sign

In [ ]:
loss.backward()
for p in mlp.parameters():
    p.data += (-0.001) * p.grad
ypred = [mlp(xi) for xi in xs]
updated_loss = sum((ygt - yout)**3 for ygt, yout in zip(ys, ypred))
print("Updated loss:", updated_loss.data)

Okay, so clearly something didn't work.

After a bit more of research, apparently this bug is caused because we must zero out the grad of our parameters before a backward pass

Makes sense, we do not want the grads of previous backward passes accumulating in our current backward pass

I'll paste here the code for everything, redefined

In [1]:
import math
from graphviz import Digraph
import random as rand

class Value:

    def __init__(self, data, _children=(), _op="", label="", has_labels=False):
        self.data = data
        self.label = label
        self._prev = set(_children)
        self._op = _op
        self.grad = 0
        self._backward = lambda: None
        self.has_labels = has_labels

    def backward(self):
        
        topo_sorted = []
        seen = set()

        def explore(node):
            if node not in seen:
                seen.add(node)
                for child in node._prev:
                    explore(child)
                topo_sorted.append(node)

        # This changes the topo_sorted list to be a topological sort of our nodes 
        explore(self)

        self.grad = 1
        for node in reversed(topo_sorted):
            node._backward()

    def create_graph(self):

        def trace(root):
            nodes, edges = set(), set()

            def build(node):
                if node not in nodes:
                    nodes.add(node)
                    for child in node._prev:
                        edges.add((child, node))
                        build(child)

            build(root)

            return nodes, edges
        
        def graph(root):
            nodes, edges = trace(root)

            dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}, filename="my_graph")

            for node in nodes:
                node_id = str(id(node))
                dot.node(name=node_id, label=f"{node.label} | data {node.data:.3f} | grad {node.grad:.3f}", shape="record")

                if node._op:
                    dot.node(name=(node_id+ node._op), label=node._op)
                    dot.edge((node_id + node._op), node_id)

            for child, parent in edges:
                dot.edge(str(id(child)), (str(id(parent)) + parent._op))

            return dot
    
        g = graph(self)
        return g

    def __repr__(self):
        return f"Value(data={self.data:.3f})" if self.label=="" else f"Value(\"{self.label}\", data={self.data:.3f})"

    def __add__(self, other):
        
        other = other if isinstance(other, Value) else Value(data=other, label=f"{other}")

        out = Value(data=(self.data + other.data), _children=(self, other), _op="+")

        l = f"{self.label} + {other.label}" if (self.label != "" and other.label != "" and self.has_labels) else ""
        out.label = l 

        def _backward():
            dout_dself = 1
            dout_dother = 1

            self.grad += out.grad * dout_dself
            other.grad += out.grad * dout_dother
        out._backward = _backward

        return out

    def __radd__(self, other):
        return self + other

    def __mul__(self, other):

        other = other if isinstance(other, Value) else Value(data=other, label=f"{other}")

        out = Value(data=(self.data * other.data), _children=(self, other), _op="*")
        
        l = f"({self.label})*{other.label}" if (self.label != "" and other.label != "" and self.has_labels) else ""
        out.label = l 

        def _backward():
            dout_dself = other.data
            dout_dother = self.data

            self.grad += out.grad * dout_dself
            other.grad += out.grad * dout_dother
        out._backward = _backward

        return out

    def __rmul__(self, other):
        return self * other
    
    def __pow__(self, other):

        assert isinstance(other, (int, float))

        out = Value(data=(self.data**other), _children=(self, ), _op=f"^{other}")

        l = f"({self.label})^{other}" if (self.label != "" and self.has_labels) else ""
        out.label = l

        def _backward():
            dout_dself = other * self.data**(other - 1)
            self.grad += out.grad * dout_dself
        out._backward = _backward

        return out
    
    def __truediv__(self, other):
        return self * other**-1
    
    def __rtruediv__(self, other):
        return self * other**-1
    
    def __neg__(self):
        return self * -1
    
    def __sub__(self, other):
        return self + (-other)

    def __rsub__(self, other):
        return self + (-other)

    def raise_euler(self):
        out = Value(data=math.exp(self.data), _children=(self, ), _op=f"e^{self.data}")

        l = f"e^({self.label})" if (self.label != "" and self.has_labels) else ""
        out.label = l

        def _backward():
            dout_dself = math.exp(self.data)
            self.grad += out.grad * dout_dself
        out._backward = _backward

        return out
    
    # tanh(x) = (e^(2x) - 1) / (e^(2x) + 1)
    def tanh(self):
        out = Value(data=math.tanh(self.data), _children=(self, ), _op="tanh")

        l = f"tanh^({self.label})" if (self.label != "" and self.has_labels) else ""
        out.label = l

        def _backward():
            dout_dself = 1 - out.data**2
            self.grad += out.grad * dout_dself

        out._backward = _backward
        return out

class Module:
    pass

class Neuron:
    def __init__(self, n_weights):
        self.weights = [Value(rand.uniform(-1, 1)) for _ in range(n_weights)]
        # self.weights = [Value(1) for _ in range(n_weights)]

        self.bias = Value(rand.uniform(-1, 1))
        # self.bias = Value(1)

    def __call__(self, inputs):
        activation = sum((input_i * weight_i for input_i, weight_i in zip(inputs, self.weights)), self.bias)
        out = activation.tanh()
        return out

    def parameters(self):
        return [self.bias] + self.weights

    def __repr__(self):
        name = ""
        w = self.weights[0]
        start = w.label.index("N")
        name = w.label[start:]
        return f"Neuron(w={[round(w.data, 3) for w in self.weights]}, b={round(self.bias.data, 3)}, name={name})"

class Layer:
    def __init__(self, n_weights, n_neurons):
        self.neurons = [Neuron(n_weights) for _ in range(n_neurons)]
    
    def __call__(self, inputs):
        outs = [n(inputs) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

    def __repr__(self):
        return f"Layer({self.neurons})"

class MLP:
    def __init__(self, n_weights_per_neuron, n_neurons_per_layer):
        size = [n_weights_per_neuron] + n_neurons_per_layer
        n_layers = len(n_neurons_per_layer)
        self.layers = [Layer(n_weights=size[i], n_neurons=size[i+1]) for i in range(n_layers)]

        # Use this alongside has_labels=True if you want something cursed
        for layer_index, layer in enumerate(self.layers):
            for neuron_index, neuron in enumerate(layer.neurons):
                for weight_index, weight in enumerate(neuron.weights):
                    weight.label = f"W{weight_index}-N{neuron_index}-L{layer_index}"
                neuron.bias.label = f"B-N{neuron_index}-L{layer_index}"

    def __call__(self, inputs):
        for l in self.layers:
            inputs = l(inputs)

        return inputs 

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

    def __repr__(self):
        ls = ""

        for l in self.layers:
            ls = ls + f"{l}\n"

        output = "MLP\n" + ls

        return output
        

In [2]:
# Define the MLP
n_inputs = 3
mlp = MLP(n_inputs, [4, 4, 1])

# Print information about the MLP
# print(mlp)
# params = mlp.parameters()
# print(f"Total of {len(params)} parameters.")
# for p in mlp.parameters():
#     print(p)

# Define the training data-set
inputs = [
    [Value(2.0, label="1"), Value(3.0, label="2"), Value(-1.0, label="3")],
    [Value(3.0, label="4"), Value(-1.0, label="5"), Value(0.5, label="6")],
    [Value(0.5, label="7"), Value(1.0, label="8"), Value(1.0, label="9")],
    [Value(1.0, label="10"), Value(1.0, label="11"), Value(-1.0, label="12")],
]
utopian_outputs = [Value(1.0, label="pred1"), Value(-1.0, label="pred2"), Value(-1.0, label="pred3"), Value(1.0, label="pred4")]
before = [mlp(input_) for input_ in inputs]



In [3]:
# Create the training loop
for k in range(1000):

    # Forward pass
    # We have updated the parameters of our network.
    # Hence, our actual_outputs must have changed
    # And hence our loss must too have changed
    actual_outputs = [mlp(input_) for input_ in inputs]
    loss = sum((utopian_output - actual_output)**2 for utopian_output, actual_output in zip(utopian_outputs, actual_outputs))

    # Zero-grad
    # Basically flushing out all of the previous gradients
    # Because they only tell us how to influence the loss using our previous parameters,
    # and not utilizing the updated parameters of the previous loop
    for p in mlp.parameters():
        p.grad = 0.0

    # Backward pass
    # Update our info on how to influence the loss
    loss.backward()

    # Decrease the loss
    for p in mlp.parameters():
        p.data += (-0.01) * p.grad

    # Display the decreased loss
    # print(k, loss.data)

In [5]:
# Display our network before and after training
after = actual_outputs
print(f"Before training: {before}")
print(f"After training:  {after}")

Before training: [Value(data=0.379), Value(data=0.332), Value(data=0.635), Value(data=-0.150)]
After training:  [Value(data=0.980), Value(data=-0.978), Value(data=-0.966), Value(data=0.968)]
